# Board SYZYGY Interface
----

## Aim/s
* Enabling the SYZYGY interface on the board.
* Test the SYZYGY [SZG-TST-STD](https://docs.opalkelly.com/display/SZGPODS/SZG-TST-STD) module (shown below).


## References
* [SYZYGY documentation](http://syzygyfpga.io/wp-content/uploads/2019/09/Syzygy-DNA-Specification-V1p1.pdf)
* [SZG-TST-STD Test](https://docs.opalkelly.com/display/SZGPODS/SZG-TST-STD)

## Last revision
* 27Jan21 - Initial revision
----

## Pre-test setup

#### SYZYGY VIO pin supported voltages
The supported voltages for the VIO pin of the SYZYGY interface can be:
* 1.2V
* 1.8V
* 2.5V
* 3.3V

<div class="alert alert-box alert-danger">
    The SYZYGY interface does <strong>not</strong> support hot-plug. <a href="http://syzygyfpga.io/wp-content/uploads/2019/09/Syzygy-DNA-Specification-V1p1.pdf">SYZYGY documentation</a>.
</div>

#### Before running the test, the following steps should be completed:
* Plugged the SZG-TST-STD module onto the board.
* Boot the board
* Setup voltage to **1.8V** (see below).
</div>

<img src="https://opalkelly.com/wp-content/uploads/TST-POD-STD.png" alt="SZG-TST-STD" style="width: 700px;"/>

### Load overlay

In [ ]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay("base.bit")

Now let's set the VIO voltage to be 1.8V.

In [ ]:
base.set_syzygy_vio(1.8)

You should be able to verify that the voltage on VIO point of the SZG-TST-STD module is 1.8V.

### Test the SYZYGY-TST-STD module

On base overlay, the SYZYGY data pins are connected to PL AXI GPIO block.
So we can perform some GPIO operations to verify the data path.

For the SYZYGY-TST-STD module, the mapping between AXI GPIO and the module
pin layout is shown below.

| GPIO pin | Module pin || GPIO pin | Module pin |
|----------|------------||----------|------------|
| 0        | S0         ||          |            |
| 2        | S2         ||          |            |
| 4        | S4         ||          |            |
| 6        | S6         ||          |            |
| 1        | S1         || 3        | S3         |
| 5        | S5         || 7        | S7         |
| 8        | S8         || 10       | S10        |
| 9        | S9         || 11       | S11        |
| 12       | S12        || 14       | S14        |
| 13       | S13        || 20       | S15        |
| 16       | S16        || 18       | S18        |
| 17       | S17        || 19       | S19        |
| 20       | S20        || 22       | S22        |
| 21       | S21        || 23       | S23        |
| 24       | S24        || 26       | S26        |
| 25       | S25        || 27       | S27        |
| 28       | P2C_CLKp   || 30       | P2C_CLKn   |
| 29       | C2P_CLKp   || 31       | C2P_CLKn   |


In the above table, keep in mind that the module pins on the same row
are connected as a loop-back on the module. For example, if you write a
logic 1 to pin S1, then you are supposed to get logic 1 at pin S3, 
and vice-versa. 

Based on this mapping, we can set each GPIO pin to be input (1) or output (0).

In [ ]:
base.syzygy_std0.write(4, 0xCCCCCC9D)

We can first check the power supply is good or not. Based on the data sheet,
supply status is communicated back through SYZYGY pins S0, S2, S4, and S6.
The status is shown below:

|         | Power | 5.5V   | 3.3V   | VIO    |
|---------|-------|--------|--------|--------|
| S6 = 1  | Good  | S0 = 1 | S4 = 1 | S2 = 1 |
|         | Bad   | S0 = 0 | S4 = 0 | S2 = 0 |
| S6 = 0  | Good  | S0 = 0 | S4 = 0 | S2 = 0 |
|         | Bad   | S0 = 1 | S4 = 1 | S2 = 1 |

In [ ]:
base.syzygy_std0.write(0, 0x40)
assert base.syzygy_std0.read(0) & 1 == 1, "5.5V power is bad."
assert base.syzygy_std0.read(0) & 4 == 4, "VIO power is bad."
assert base.syzygy_std0.read(0) & 16 == 16, "3.3V power is bad."
base.syzygy_std0.write(0, 0)
assert base.syzygy_std0.read(0) & 1 == 0, "5.5V power is bad."
assert base.syzygy_std0.read(0) & 4 == 0, "VIO power is bad."
assert base.syzygy_std0.read(0) & 16 == 0, "3.3V power is bad."
print("Power test passed.")

Before we check all the pin pairs, let's define a test function.

In [ ]:
def test_loop_back(pin_pair):
    input_pin, output_pin = pin_pair
    write_value = 0x1 << input_pin
    base.syzygy_std0.write(0, write_value)
    read_value = base.syzygy_std0.read(0)
    assert (
        read_value & (0x1 << output_pin)
    ) >> output_pin == 0x1, "Read value is {}.".format(read_value)
    write_value = 0x0
    base.syzygy_std0.write(0, write_value)
    read_value = base.syzygy_std0.read(0)
    assert (
        read_value & (0x1 << output_pin)
    ) >> output_pin == 0x0, "Read value is {}.".format(read_value)

We can check all the pairs now.

In [ ]:
pin_pairs = [
    (1, 3),
    (5, 7),
    (8, 10),
    (9, 11),
    (12, 14),
    (13, 15),
    (16, 18),
    (17, 19),
    (20, 22),
    (21, 23),
    (24, 26),
    (25, 27),
    (28, 30),
    (29, 31),
]
_ = [test_loop_back(i) for i in pin_pairs]
print("Pin pair loop-back tests passed.")

<div class="alert alert-box alert-success">
    Now we have confidence that the SYZYGY modules work on this board.
</div>

---
Copyright (C) 2022 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----